In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [31]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df.shape

(3066766, 19)

3066766 will be use in Q3

In [32]:
print("How many columns are there?")
print(f"{df.shape[1]} columns")

How many columns are there?
19 columns


## Q2. Computing duration

In [39]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df['duration']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [40]:
print("What's the standard deviation of the trips duration in January?")
print(df['duration'].std())

What's the standard deviation of the trips duration in January?
42.59435124195458


## Q3. Dropping outliers

In [41]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.shape

(3009173, 20)

In [44]:
print("What fraction of the records left after you dropped the outliers?")
print(3009173 / 3066766 * 100)

What fraction of the records left after you dropped the outliers?
98.1220282212598


## Q4. One-hot encoding

In [45]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [47]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [55]:
X_train.get_shape()

(3009173, 516)

In [56]:
print("What's the dimensionality of this matrix (number of columns)?")
print(X_train.get_shape()[1])

What's the dimensionality of this matrix (number of columns)?
516


## Q5. Training a model

In [57]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

c:\Users\FrancoRamos\Videos\mlops-zoomcamp\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.658405703368252

In [58]:
print("What's the RMSE on train?")
print(mean_squared_error(y_train, y_pred, squared=False))

What's the RMSE on train?
7.658405703368252


c:\Users\FrancoRamos\Videos\mlops-zoomcamp\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Q6. Evaluating the model

In [63]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [64]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [65]:
len(df_train), len(df_val)

(3009173, 2855951)

In [66]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [70]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [71]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [73]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

c:\Users\FrancoRamos\Videos\mlops-zoomcamp\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.658405703368252

In [74]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\FrancoRamos\Videos\mlops-zoomcamp\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


12.509284386693556

In [75]:
print("What's the RMSE on validation?")
print(mean_squared_error(y_val, y_pred, squared=False))

What's the RMSE on validation?
12.509284386693556


c:\Users\FrancoRamos\Videos\mlops-zoomcamp\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
